In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

In [20]:
# 1. Load and prepare the data
df = pd.read_excel(r"C:\Users\Dr. Gonzo\OneDrive\Desktop\001. Finance AI Agents 2026\Sales_3y_CLEANED.xlsx")
df['Date'] = pd.to_datetime(df['Date'])

# Monthly aggregation
monthly = (df.groupby(df['Date'].dt.to_period('M'))
             .agg({'Revenue': 'sum', 'Cost': 'sum', 'Quantity': 'sum'})
             .round(0)
             .reset_index())
monthly['Date'] = monthly['Date'].dt.to_timestamp()
monthly = monthly.sort_values('Date').reset_index()

In [21]:
#2 Dash App Setup
app = Dash(__name__, title="Sales Performance Dashboard")

app.layout = html.Div(style={'backgroundColor': '#111111', 'color': 'white', 'fontFamily': 'Arial'}, children=[
    html.H1("Sales Performance Dashboard", style={'textAlign': 'center', 'margin': '30px 0', 'color': '#00CC96'}),

    # View Mode Dropdown
    html.Div([
        html.Label("View Mode", style={'fontSize': 20, 'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='view-mode',
            options=[
                {'label': 'Actuals Only',                'value': 'actuals'},
                {'label': 'Actuals vs Previous Year',    'value': 'vs_py'},
                {'label': 'Actuals vs Budget (coming soon)', 'value': 'budget', 'disabled': True}
            ],
            value='actuals',
            clearable=False,
            style={'width': '400px', 'margin': '20px auto'}
        )
    ], style={'textAlign': 'center'}),

    # Metric Checklist
    html.Div([
        html.Label("Select Metrics:", style={'fontSize': 18, 'marginLeft': 20}),
        dcc.Checklist(
            id='metric-checklist',
            options=[
                {'label': ' Revenue', 'value': 'Revenue'},
                {'label': ' OpEx (Cost)', 'value': 'Cost'},
                {'label': ' Quantity Sold', 'value': 'Quantity'}
            ],
            value=['Revenue', 'Cost'],
            inline=True,
            style={'margin': '10px 20px', 'fontSize': 17}
        )
    ]),

    dcc.Graph(id='main-chart'),

    # Date Range Slider
    html.Div([
        html.H4("Date Range", style={'textAlign': 'center'}),
        dcc.RangeSlider(
            id='date-slider',
            min=monthly['Date'].min().timestamp(),
            max=monthly['Date'].max().timestamp(),
            value=[monthly['Date'].min().timestamp(), monthly['Date'].max().timestamp()],
            marks={int(ts): pd.Timestamp(ts, unit='s').strftime('%Y-%m')
                   for ts in monthly['Date'][::6].view('int64') // 10**9},
            step=30*24*60*60,
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'width': '90%', 'margin': '40px auto'})
])

C:\Users\Dr. Gonzo\AppData\Local\Temp\ipykernel_1084\575140882.py:50: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.



In [22]:
# 3. Callback to update the chart
@app.callback(
    Output('main-chart', 'figure'),
    [Input('view-mode', 'value'),
     Input('metric-checklist', 'value'),
     Input('date-slider', 'value')]
)
def update_chart(view_mode, selected_metrics, date_range):
    start_date = pd.to_datetime(date_range[0], unit='s')
    end_date   = pd.to_datetime(date_range[1], unit='s')
    
    filtered = monthly[(monthly['Date'] >= start_date) & (monthly['Date'] <= end_date)].copy()

    # Prepare Previous Year data if needed
    py_data = None
    if view_mode == 'vs_py':
        py = filtered.copy()
        py['Date'] = py['Date'] + pd.DateOffset(years=1)
        py = py[py['Date'] <= monthly['Date'].max()]
        py_data = py

    fig = go.Figure()

    colors = {'Revenue': '#00CC96', 'Cost': '#EF553B', 'Quantity': '#636EFA'}

    # Main traces
    if 'Revenue' in selected_metrics:
        fig.add_trace(go.Scatter(x=filtered['Date'], y=filtered['Revenue'],
                                 mode='lines+markers', name='Revenue',
                                 line=dict(color=colors['Revenue'], width=4),
                                 hovertemplate='Revenue: $%{y:,.0f}'))
        if py_data is not None:
            fig.add_trace(go.Scatter(x=py_data['Date'], y=py_data['Revenue'],
                                     mode='lines', name='Revenue PY',
                                     line=dict(color=colors['Revenue'], dash='dot', width=2),
                                     opacity=0.6))

    if 'Cost' in selected_metrics:
        fig.add_trace(go.Scatter(x=filtered['Date'], y=filtered['Cost'],
                                 mode='lines+markers', name='OpEx (Cost)',
                                 line=dict(color=colors['Cost'], width=4),
                                 hovertemplate='OpEx: $%{y:,.0f}'))
        if py_data is not None:
            fig.add_trace(go.Scatter(x=py_data['Date'], y=py_data['Cost'],
                                     mode='lines', name='OpEx PY',
                                     line=dict(color=colors['Cost'], dash='dot', width=2),
                                     opacity=0.6))

    if 'Quantity' in selected_metrics:
        fig.add_trace(go.Scatter(x=filtered['Date'], y=filtered['Quantity'],
                                 mode='lines+markers', name='Quantity Sold',
                                 line=dict(color=colors['Quantity'], width=4),
                                 yaxis='y2',
                                 hovertemplate='Qty: %{y:,.0f}'))
        if py_data is not None:
            fig.add_trace(go.Scatter(x=py_data['Date'], y=py_data['Quantity'],
                                     mode='lines', name='Qty PY',
                                     line=dict(color=colors['Quantity'], dash='dot', width=2),
                                     opacity=0.6, yaxis='y2'))

    # Layout
    fig.update_layout(
        title="Revenue, OpEx & Quantity Over Time",
        xaxis_title="Date",
        yaxis_title="Amount ($)",
        template="plotly_dark",
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=650,
        margin=dict(l=60, r=60, t=100, b=60)
    )

    # Secondary axis for Quantity
    if 'Quantity' in selected_metrics:
        fig.update_layout(
            yaxis2=dict(
                title="Quantity Sold",
                overlaying="y",
                side="right",
                showgrid=False,
                tickfont=dict(color=colors['Quantity'])
            )
        )

    return fig

In [23]:
if __name__ == '__main__':
    print("Dashboard launching... Opening in your browser!")
    app.run(debug=True)

Dashboard launching... Opening in your browser!
